In [13]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
import math

In [14]:
def StudentDistribution_px(x, m):
    return m**(m/2) * math.gamma((m+1)/2) / (np.sqrt(np.pi) * math.gamma(m/2) * (x**2 + m)**((m+1)/2))
def NormDistribution_px(x, a, s2):
        return (1/np.sqrt(2 * np.pi * s2) * np.exp(-1 * (x - a)**2 / (2*s2)))

def NormDistribution_Fx(x, a, s2):
    i, e = quad(NormDistribution_px, -np.inf, x, args = (a,s2))
    return i

def FisherDistribution_px(x, n, m):
    return n**(n/2) * m**(m/2) * x**(n/2 - 1) * math.gamma((n+m)/2) / ( (m+n*x)**((n+m)/2) * math.gamma(n/2) * math.gamma(m/2) )

def F_empir(v, prom):
    n = len(v)
    x = prom
    F_tilda = np.array([len(np.where(v < i)[0])/n for i in x])
    return F_tilda

In [15]:
def regression_analysis(PSI, Y):
    
    n, p = PSI.shape

    F = np.dot(np.transpose(PSI), PSI)
    Fisher = np.linalg.inv(F) 
    betta_coefficient = np.dot(Fisher, np.dot(np.transpose(PSI),Y))
    
    e = Y - np.dot(PSI, betta_coefficient)
    RSS = np.dot(np.transpose(e), e)[0][0]
    
    betta_significance_coefficient = np.zeros(p)
    for i in range(p):
        betta_significance_coefficient[i] = betta_coefficient[i][0] * np.sqrt(n-p) / np.sqrt(RSS * Fisher[i,i])
    
    betta_p_value = np.zeros(p)
    for i in range(p):
        betta_p_value[i], _ = quad(StudentDistribution_px, betta_significance_coefficient[i], +np.inf, args=(n-p))
        betta_p_value[i] *= 2
    
    
    TSS = ((Y - Y.mean())**2).sum()
    R2 = 1 - RSS/TSS
    err = np.transpose(e)[0]

    delta_regr = R2/(1 - R2) * (n-p)/(p-1)
    regression_p_value, _ = quad(FisherDistribution_px, delta_regr, +np.inf, args=(p-1, n-p))
    
    return betta_coefficient, betta_p_value, err, Fisher, RSS, TSS, R2, regression_p_value

# T2

In [16]:
group_to_persents = {1: np.array([83,85]), 2: np.array([64, 85, 85, 86, 86, 87])}
group_to_persents[3] = np.array([86, 87, 87, 87, 88, 88, 88, 88, 88, 89, 90])
group_to_persents[4] = np.array([89, 90, 90, 91])
group_to_persents[5] = np.array([90, 92])
groups = np.array([1,2,3,4,5])
for gr in groups:
    print(f"{gr} : {group_to_persents[gr]} : {len(group_to_persents[gr])}")

1 : [83 85] : 2
2 : [64 85 85 86 86 87] : 6
3 : [86 87 87 87 88 88 88 88 88 89 90] : 11
4 : [89 90 90 91] : 4
5 : [90 92] : 2


## a) 

In [17]:
n = sum(len(group_to_persents[gr]) for gr in groups)
p = len(groups)

PSI = np.zeros((p,n), dtype=int)
for i in range(p):
    b = sum(len(group_to_persents[gr]) for gr in groups[:i])
    e = sum(len(group_to_persents[gr]) for gr in groups[:i+1])
    PSI[i,b:e] = 1
PSI = np.transpose(PSI)

Y = []
for i in groups:
    for ps in group_to_persents[i]:
        Y.append(ps)
Y = np.transpose(np.array([Y]))

np.concatenate((PSI,Y), axis=1)

array([[ 1,  0,  0,  0,  0, 83],
       [ 1,  0,  0,  0,  0, 85],
       [ 0,  1,  0,  0,  0, 64],
       [ 0,  1,  0,  0,  0, 85],
       [ 0,  1,  0,  0,  0, 85],
       [ 0,  1,  0,  0,  0, 86],
       [ 0,  1,  0,  0,  0, 86],
       [ 0,  1,  0,  0,  0, 87],
       [ 0,  0,  1,  0,  0, 86],
       [ 0,  0,  1,  0,  0, 87],
       [ 0,  0,  1,  0,  0, 87],
       [ 0,  0,  1,  0,  0, 87],
       [ 0,  0,  1,  0,  0, 88],
       [ 0,  0,  1,  0,  0, 88],
       [ 0,  0,  1,  0,  0, 88],
       [ 0,  0,  1,  0,  0, 88],
       [ 0,  0,  1,  0,  0, 88],
       [ 0,  0,  1,  0,  0, 89],
       [ 0,  0,  1,  0,  0, 90],
       [ 0,  0,  0,  1,  0, 89],
       [ 0,  0,  0,  1,  0, 90],
       [ 0,  0,  0,  1,  0, 90],
       [ 0,  0,  0,  1,  0, 91],
       [ 0,  0,  0,  0,  1, 90],
       [ 0,  0,  0,  0,  1, 92]])

In [18]:
betta_coefficient, betta_p_value, err, Fisher, RSS, TSS, R2, regression_p_value = regression_analysis(PSI, Y)

In [19]:
Fisher

array([[0.5       , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.16666667, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.09090909, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.25      , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.5       ]])

In [20]:
alpha = 0.05
for i in range(p):
    print("betta[{}] = {:3f}, p-value = {:3f}".format(i, betta_coefficient[i][0], betta_p_value[i]), end=" ")
    if betta_p_value[i] <= alpha :
        print("коэффициент значимо отличен от нуля")
    else:
        print("коэффициент не значимо отличен от нуля")
    

betta[0] = 84.000000, p-value = 0.000000 коэффициент значимо отличен от нуля
betta[1] = 82.166667, p-value = 0.000000 коэффициент значимо отличен от нуля
betta[2] = 87.818182, p-value = 0.000000 коэффициент значимо отличен от нуля
betta[3] = 90.000000, p-value = 0.000000 коэффициент значимо отличен от нуля
betta[4] = 91.000000, p-value = 0.000000 коэффициент значимо отличен от нуля


In [21]:
RSS, TSS, R2, regression_p_value

(416.469696969697, 648.56, 0.3578547906597739, 0.05458335566236725)

In [25]:
alpha = 0.05
if regression_p_value <= alpha:
    print("регрессия значима")
else:
    print("нет оснований для отклонения гипотезы ")

нет оснований для отклонения гипотезы 


## b) 

In [24]:
alpha = 0.05
for i in np.arange(p):
    for j in np.arange(i+1,p):
        delta_tilda = (betta_coefficient[i] - betta_coefficient[j]) / np.sqrt(RSS * (Fisher[i,i] + Fisher[j, j])) * np.sqrt(n-p)
        p_value, _ = quad(StudentDistribution_px, np.abs(delta_tilda), +np.inf, args=(n-p))
        p_value *= 2
        print("p-value = {: 2f}".format(i, j, p_value))
        if p_value <= alpha:
            print(f"группы {i} и {j} различаются")
        else:
            print(f"группы {i} и {j} одинаковы")

p-value =  0.000000
группы 0 и 1 одинаковы
p-value =  0.000000
группы 0 и 2 одинаковы
p-value =  0.000000
группы 0 и 3 одинаковы
p-value =  0.000000
группы 0 и 4 одинаковы
p-value =  1.000000
группы 1 и 2 различаются
p-value =  1.000000
группы 1 и 3 различаются
p-value =  1.000000
группы 1 и 4 различаются
p-value =  2.000000
группы 2 и 3 одинаковы
p-value =  2.000000
группы 2 и 4 одинаковы
p-value =  3.000000
группы 3 и 4 одинаковы
